In [81]:
import pandas as pd
import config.config as config
from preprocessing.preprocessors import get_meta_dict


ImportError: cannot import name 'get_meta_dict' from 'preprocessing.preprocessors' (/Users/matthewgalloway/Repos/dataiku_interview/preprocessing/preprocessors.py)

In [79]:
data = pd.read_csv(config.TRAIN_DIR, names=get_meta_columns())
data.head()

NameError: name 'get_meta_columns' is not defined